Cross-validation is a crucial technique in assessing the performance of machine learning models. K-fold cross-validation, a widely-used method, involves dividing the dataset into K subsets, training the model K times, each time using a different subset as the testing set. This helps us gauge how well our model generalizes to unseen data. However, as K increases so does the computational time. This becomes painfully evident, particularly during hyperparameter tuning, where sluggish fits can be a major bottleneck.

 Leave-one-out cross-validation (LOOCV), a special case of K-fold cross-validation where K equals the number of training samples, can offer  accurate evaluation but comes at a hefty computational cost, making it less practical for larger datasets and hyperparameter tuning.

For linear models like ordinary least squares and ridge regression, a little-known trick exists to efficiently calculate LOOCV scores. scikit-learn even implements this in it's `RidgeCV` estimator. Notably, this same trick extends beyond these linear models to any problem involving quadratically regularized least squares regression—a fact not widely recognized.

Taking it a step further, even for non-least-squares models like logistic and Poisson regression, a similar trick can be employed to approximate LOOCV scores efficiently. Intriguingly, the accuracy of this approximation improves with larger datasets, addressing the need for speedup in precisely those scenarios.

In this blog post, we derive the formulas for efficient LOOCV calculations and demonstrate these results on a practical example.

# Notation
We use $n$ to denote the number of samples in the training dataset.

The $m$-dimensional feature vectors are represented as $x_1$ to $x_n$, forming the rows of matrix $X$.

Targets are denoted as $y_1$ to $y_n$, forming the vector $y$. The model's prediction for the $i$-th training sample is $\hat{y}_i = x_i^T \theta$, where $\theta$ is the vector of coefficients.
$\hat{y} = X \theta$ is the vector containing all predictions.

We fit $\theta$ to the training data by minimizing the combined loss and regularization terms:
$$
\theta := \arg\min_{\theta'} f(\theta').
$$ {#eq-theta-def}
where
$$
f(\theta') := \sum_{i=1}^{n} l(x_i^T \theta'; y_i) + r(\theta').
$$
Here, $l(\hat{y}_i; y_i)$ represents the loss function, quantifying the difference between the prediction $\hat{y}$ and the true target $y_i$,
while $r$ is the regularization function.
We will assume $l$ (as a function of $\hat{y}_i$) and $r$ are strictly convex and twice differentiable.
Special cases of this model includes 
ordinary least squares ($l(\hat{y}_i; y_i) = (\hat{y}_i - y_i)^2$, $r(\theta') = 0$), 
ridge regression ($l(\hat{y}_i; y_i) = (\hat{y}_i - y_i)^2$, $r(\theta') = 0$), 
logistic regression ($l(\hat{y}_i;y_i) = \log \left( 1 + e^{-y_i \hat{y}_i}\right)$ with $y_i \in \{ -1, 1\}$),
and Poisson regression ($l(\hat{y}_i;y_i) = y_i \hat{y}_i - e^{\hat{y}_i}$).

To denote the coefficients obtained by excluding the $j$-th example, we use $\theta^{(j)}$:
$$
\theta^{(j)} = \arg\min_{\theta'} f^{(j)} (\theta')
$$
where
$$ f^{(j)}(\theta') := \sum_{i \neq j} l(x_i^T \theta'; y_i) + r(\theta') $$
Similarly, $X^{(j)}$ and $y^{(j)}$, are employed to represent $X$ and $y$ with the $j$-th row removed, respectively.
We denote by $\tilde{y}_j$ the predicted label for sample $j$ when it is left out:
$$
\tilde{y}_j := x_j ^T \theta^{(j)}
$$ {#eq-y-tilde-j-def}
Our goal is calculating $\tilde{y}_j$, for all $j$, efficiently.


# LOOCV with the sum of squares loss and quadratic regularization
In scenarios where the loss function is the sum of squares loss
$$
l(\hat{y}_i; y_i) = (\hat{y}_i - y_i)^2,
$$
and the regularizer is quadaratic
$$
r(\theta') = \theta'^T R \theta'
$$
where $R$ is an $m \times m$ semi-positive definite matrix,
the solution to optimization problem @eq-theta-def is obtained by solving the linear equation [^1]:
$$ 
A \theta = b. 
$$ {#eq-theta-solve}
where
\begin{align*}
    A &:= X^T X + R \\
    b &:= X^T y.
\end{align*}

[^1]: I am deliberately avoiding writing $\theta = A^{-1} b$, as $A$ does not have to be invertible for this equation to have a solution. 
Furthermore, it can mislead people into an implementation like `np.linalg.inv(A) @ b`, which is less stable and efficient than an implementation like `np.linalg.solve(A, b)`.

Similarly, obtaining $\theta^{(j)}$ requires solving
$$ 
A^{(j)} \theta^{(j)} = b^{(j)}.
$$ {#eq-theta-j-solve}
where
\begin{align*}
    A^{(j)} &:= X^{(j)T} X^{(j)} + R \\
    b^{(j)} &:= X^{(j)T} y^{(j)}.
\end{align*}
Solving this linear equation for each $j$ has a time complexity of $O(m^2 n)$ (assuming $n > m$).
Therefore, in a naive implementation, the overall complexity of LOOCV becomes $O(m^2 n^2)$, posing a significant computational challenge, particularly when $n$ is large.

The key idea behind efficient LOOCV lies in leveraging the solution for @eq-theta-solve to calculate the solution for each @eq-theta-j-solve.
We will utilize an important idea from computational linear algebra: the time required to solve multiple $m$ by $m$ equations that share the same matrix is almost identical to the time it takes to solve a single $m$ by $m$ equation.
Specifically, we will solve, in additional to @eq-theta-solve, the following $n$ equations:
$$
A t_j = x_j.
$$

We start by noting that
\begin{align*}
X^TX &= X^{(j)^T} X^{(j)} + x_j x_j^T    \\
X^Ty &= X^{(j)^T} y^{(j)} + x_j y_j,
\end{align*}
so we can write @eq-theta-j-solve like so:
$$
(A - x_j x_j^T) \theta^{(j)} = b - x_j y_j.
$$
The usual way forward is employing Sherman-Morrison formula, solve for $\theta^{(j)}$ and substitute in @eq-y-tilde-j-def to get an expression for $\tilde{y}$.
But there is a better way [^2]: rewrite @eq-theta-j-solve as:
\begin{align*}
    A \theta^{(j)} - x_j \tilde{y}_j &= b - x_j y_j \\
    \tilde{y}_j &= x_j ^T \theta^{(j)}
\end{align*}
so instead of a single equation with one unknown ($\theta^{(j)}$),
we now have two equations with two 
 unknowns ($\theta^{(j)}$ and $\tilde{y}_j$).
 At first this seems more complicated, but notice that since the coefficient of $\theta^{(j)}$ in the first equation is $A$, we can eliminate it:
\begin{align*}
\theta^{(j)}  &= A^{-1} ( b - x_j y_j + x_j \tilde{y}_j ) \\
&= \theta - t_j (  y_j - \tilde{y}_j )
\end{align*}
substituting in the bottom equation, we can solve for $\tilde{y}_j$:
\begin{align*}
\tilde{y}_j &= x_j ^T \left( \theta - t_j (  y_j - \tilde{y}_j ) \right)
\\
\tilde{y}_j &= \hat{y}_j - h_j (y_j - \tilde{y}_j)
\\
\tilde{y}_j &= \frac{\hat{y}_j - h_j y_j}{1-h_j}
% \\
% \tilde{y}_j &= \frac{\hat{y}_j -h_j \hat{y}_j + h_j \hat{y}_j - h_j y_j}{1-h_j}
\\
\tilde{y}_j &= \hat{y}_j + \frac{h_j }{1-h_j} \left( \hat{y}_j - y_j \right)
% \\
% \tilde{y}_j &= \frac{\hat{y}_j - y_j}{1-h_j} + y_j
\end{align*}
where 
$$
h_j := x_j ^T t_j = x_j^T A^{-1} x_j.
$$
That's it! we got an expression for $\tilde{y}_j$ that doesn't require inverting any matrix other than $A$.
It also has a nice interpretation: the difference between the prediction and the LOO prediction is the difference between the prediction an the true target, amplified by $\frac{h_j }{1-h_j}$.

If we wanted, we could substitute $\hat{y}_j$ in (5) to get a concrete expression for $\theta^{(j)}$. Although it can be useful for stuff like sensitivity analysis of the model coefficients, we won't pursuit this direction here.

[^2]: For several reasons: Translates better into code; 
We get the expression for $\tilde{y}_j$ directly, without goind through an expression for $\theta^{(j)} first;
Sherman-Morisson is a bit too strong here and can obscure some insights, so it's nice to avoid it. But actually the other way is just halfway it's proof (see for example [here](https://www.cs.cornell.edu/~bindel/class/cs6210-f09/lec12.pdf)).